In [1]:
import pandas as pd
import seaborn as sns

In [2]:
df_train = pd.read_csv("train.csv")
X_test = pd.read_csv("test.csv")

In [3]:
X_train, Y_train = df_train.drop(columns="Survived"), df_train["Survived"]

KeyError: "['Survived'] not found in axis"

## Exploration des données 

Comparaison des jeux de données 

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
X_train.isna().sum()

In [ ]:
X_test.isna().sum()

J'ai 3 colonnes qui contiennent des NAN, il va falloir que je les remplace

In [ ]:
Y_train.value_counts().plot(kind="pie")

In [ ]:
sns.violinplot(X_train["Age"])

In [ ]:
sns.violinplot(X_test["Age"])

Visiblement la distribution des Ages est assez identique entre les deux jeux de données

## Preprocessing

On supprime les colonnes pas intéressantes

In [ ]:
def drop_columns(df, columns_to_drop):
    for column in columns_to_drop:
        df.drop(columns=[column], axis=1, inplace=True)
    return df
    

In [ ]:
columns_to_drop = ["Name", "Ticket", "Cabin"]
X_train = drop_columns(X_train, columns_to_drop)
X_test =  drop_columns(X_test, columns_to_drop)

On numérise les données 

In [ ]:
def get_dummies_df(df, columns_to_dummies):
    for column in columns_to_dummies:
        if column in df.columns:
            _df = pd.get_dummies(df[column])
            df = pd.concat([df, _df], axis=1, sort=False)
            df.drop(columns=[column], inplace=True)
    return df

In [ ]:
columns_to_dummies = ["Sex", "Embarked"]
X_train = get_dummies_df(X_train, columns_to_dummies)
X_test =  get_dummies_df(X_test, columns_to_dummies)

remplacer les valeurs NAN pour les deux datasets:

In [ ]:
def replace_nan(df, Nan_columns):
    for column in Nan_columns:
        if column in df.columns:
            df[Nan_columns] = df[Nan_columns].fillna(df[Nan_columns].mean())
    return df

In [ ]:
Nan_columns = ["Age", "Fare"]
X_train = replace_nan(X_train, Nan_columns)
X_test =  replace_nan(X_test, Nan_columns)

On standardize les données

In [ ]:
# from sklearn import preprocessing
# from sklearn.preprocessing import StandardScaler
# def standardize_df(df, method):
#     columns = df.columns
#     x = df.values 
#     x_scaled = method.fit_transform(x)
#     df = pd.DataFrame(x_scaled)
#     df.columns = columns
#     return df

In [ ]:
# scaler = StandardScaler()
# X_train = standardize_df(X_train, scaler)
# X_test =  standardize_df(X_test, scaler)

# Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, Y_train)

### Prédictions

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
df_final = pd.DataFrame()

In [ ]:
df_final["PassengerId"] = X_test["PassengerId"]
df_final["Survived"] = y_pred

In [ ]:
df_final.to_csv("results.csv", index=False, sep=",")

In [ ]:
df_demo = pd.read_csv("data/train.csv")

In [ ]:
df_demo["Cabin"].astype(str).str[0:2]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV# Number of trees in random forest
import numpy as np
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
rf_random.fit(X_train, Y_train)

In [ ]:
rf_random.best_params_